In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv


In [2]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

# Data Loading

In [3]:
df_train = pd.read_csv("/kaggle/input/playground-series-s4e10/train.csv")
df_test = pd.read_csv("/kaggle/input/playground-series-s4e10/test.csv")


In [4]:
df_train.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [5]:
df_train.shape

(58645, 13)

In [6]:
df_test.shape

(39098, 12)

In [7]:
df_train.dtypes

id                              int64
person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
loan_status                     int64
dtype: object

In [8]:
df_train.dtypes

id                              int64
person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
loan_status                     int64
dtype: object

In [9]:
df_train.describe()

,id,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
count,58645.000000,58645.000000,5.864500e+04,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000,58645.000000
mean,29322.000000,27.550857,6.404617e+04,4.701015,9217.556518,10.677874,0.159238,5.813556,0.142382
std,16929.497605,6.033216,3.793111e+04,3.959784,5563.807384,3.034697,0.091692,4.029196,0.349445
min,0.000000,20.000000,4.200000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,0.000000
25%,14661.000000,23.000000,4.200000e+04,2.000000,5000.000000,7.880000,0.090000,3.000000,0.000000
50%,29322.000000,26.000000,5.800000e+04,4.000000,8000.000000,10.750000,0.140000,4.000000,0.000000
75%,43983.000000,30.000000,7.560000e+04,7.000000,12000.000000,12.990000,0.210000,8.000000,0.000000
max,58644.000000,123.000000,1.900000e+06,123.000000,35000.000000,23.220000,0.830000,30.000000,1.000000


In [10]:
df_train.drop(["id"], axis=1, inplace=True)

 # Preprocessing and Encoding Categorical Features

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## One-Hot Encoding

In [12]:
categorical_columns = ['loan_intent', 'loan_grade', 'person_home_ownership']
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(df_train[categorical_columns])
X_test_encoded = encoder.fit_transform(df_test[categorical_columns])

/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


### Encoded Columns to Dataframe

In [13]:
X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=encoder.get_feature_names_out(categorical_columns))
X_test_encoded_df = pd.DataFrame(X_test_encoded, columns=encoder.get_feature_names_out(categorical_columns))

In [14]:
X_train_encoded_df.head()

,loan_intent_DEBTCONSOLIDATION,loan_intent_EDUCATION,loan_intent_HOMEIMPROVEMENT,loan_intent_MEDICAL,loan_intent_PERSONAL,loan_intent_VENTURE,loan_grade_A,loan_grade_B,loan_grade_C,loan_grade_D,loan_grade_E,loan_grade_F,loan_grade_G,person_home_ownership_MORTGAGE,person_home_ownership_OTHER,person_home_ownership_OWN,person_home_ownership_RENT
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Concatenate Numerical Data

In [15]:
X_train_num = df_train.drop(categorical_columns + ["cb_person_default_on_file"], axis=1).reset_index(drop=True)
X_test_num = df_test.drop(categorical_columns+ ["cb_person_default_on_file"], axis=1).reset_index(drop=True)

In [16]:
X_train_num.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
0,37,35000,0.0,6000,11.49,0.17,14,0
1,22,56000,6.0,4000,13.35,0.07,2,0
2,29,28800,8.0,6000,8.90,0.21,10,0
3,30,70000,14.0,12000,11.11,0.17,5,0
4,22,60000,2.0,6000,6.92,0.10,3,0


## Standardize Numerical Data

In [17]:
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num) 
X_test_num_scaled = scaler.fit_transform(X_test_num)

### To DataFrame 

In [18]:
X_train_num_df = pd.DataFrame(X_train_num_scaled, columns=X_train_num.columns)
X_test_num_df = pd.DataFrame(X_test_num_scaled, columns=X_test_num.columns)


In [19]:
X_train_num_df.head()

,person_age,person_income,person_emp_length,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,loan_status
0,1.566200,-0.765768,-1.187200,-0.578306,0.267616,0.117378,2.031798,-0.407456
1,-0.920057,-0.212128,0.328047,-0.937775,0.880532,-0.973242,-0.946489,-0.407456
2,0.240196,-0.929223,0.833130,-0.578306,-0.585854,0.553626,1.039036,-0.407456
3,0.405947,0.156966,2.348377,0.500101,0.142396,0.117378,-0.201917,-0.407456
4,-0.920057,-0.106673,-0.682117,-0.578306,-1.238314,-0.646056,-0.698298,-0.407456


In [20]:
X_train_final = pd.concat([X_train_num_df, X_train_encoded_df], axis=1)
X_test_final = pd.concat([X_test_num_df, X_test_encoded_df], axis=1)

In [21]:
y_train = df_train['loan_status']

In [22]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_final, y_train, test_size=0.3, random_state=42)

In [23]:
print(X_train_final.head())


   person_age  person_income  person_emp_length  loan_amnt  loan_int_rate  \
0    1.566200      -0.765768          -1.187200  -0.578306       0.267616   
1   -0.920057      -0.212128           0.328047  -0.937775       0.880532   
2    0.240196      -0.929223           0.833130  -0.578306      -0.585854   
3    0.405947       0.156966           2.348377   0.500101       0.142396   
4   -0.920057      -0.106673          -0.682117  -0.578306      -1.238314   

   loan_percent_income  cb_person_cred_hist_length  loan_status  \
0             0.117378                    2.031798    -0.407456   
1            -0.973242                   -0.946489    -0.407456   
2             0.553626                    1.039036    -0.407456   
3             0.117378                   -0.201917    -0.407456   
4            -0.646056                   -0.698298    -0.407456   

   loan_intent_DEBTCONSOLIDATION  loan_intent_EDUCATION  ...  loan_grade_B  \
0                            0.0                    1.0 

# Building ANN Model 

In [24]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential()

In [25]:
#input layer
model.add(Dense(units=64, activation ="relu", input_shape=(X_train_split.shape[1],)))

#hidden layers
model.add(Dense(units=32, activation = "relu"))
model.add(Dropout(0.5))

#output layer
model.add(Dense(units=1, activation = "sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,777 (14.75 KB)

 Trainable params: 3,777 (14.75 KB)

 Non-trainable params: 0 (0.00 B)

# Training Model

In [26]:
#early stopping to stop training when validation loss stops improving
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(X_train_split, y_train_split,
                   validation_data = (X_val_split, y_val_split),
                   epochs=100, batch_size=32, callbacks=[early_stopping])
#evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(X_val_split, y_val_split)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")

Epoch 1/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9747 - loss: 0.0963 - val_accuracy: 1.0000 - val_loss: 1.5148e-05
Epoch 2/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9997 - loss: 7.0958e-04 - val_accuracy: 1.0000 - val_loss: 4.6945e-07
Epoch 3/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9997 - loss: 4.1287e-04 - val_accuracy: 1.0000 - val_loss: 1.0440e-08
Epoch 4/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9999 - loss: 2.3538e-04 - val_accuracy: 1.0000 - val_loss: 1.1818e-09
Epoch 5/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 5.6954e-05 - val_accuracy: 1.0000 - val_loss: 2.8984e-11
Epoch 6/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9998 - loss: 1.4974e-04 - val_accuracy: 1.0000 - val_loss: 3.5652e-10
Epoch 7/100
1283/1283 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 1.0000 - loss: 1.6297e-04 - val_accuracy: 1.0000 - val_loss: 2.7083e-13
Epoch 8/100
1283/1283 ━━━━━━━━━